# **Lab 2: Iterative Methods**
**Joel Widén**

# **Abstract**

Short summary of the lab report. State the objectives, methods used, main results and conlusions.

#**About the code**


This is a report in the course DD2363 Methods in Scientific Computing. The author of this file is Joel Widén, joelwid@kth.se.

# **Set up environment**

This block is run to set up the environment.

In [1]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

Numbered algorithms, equations and chapter references used in this report is from the DD2363 course book Methods in Computational Science by Johan Hoffman if not stated otherwise.

**Assignment 1:** Function: Jacobi iteration for Ax=b

* *Input:* matrix $A$, vector $b$
* *Output:* vector $x$
* *Test:* convergence of residual $|| Ax-b ||$, $|| x-y ||$ for manufactured/exact solution $y$

**Assignment 2:** Function: Gauss-Seidel iteration for $Ax=b$

* *Input:* matrix $A$, vector $b$
* *Output:* vector $x$
* *Test:* convergence of residual $|| Ax-b ||$, $|| x-y ||$ for manufactured/exact solution $y$

**Assignment 3:** Function: Newton's method for scalar nonlinear equation $f(x)=0$

* *Input:* scalar function $f(x)$
* *Output:* real number $x$
* *Test:* convergence of residual $|f(x)|$, $|x-y|$ for manufactured/exact solution $y$

# **Method**

Methods and code here

In [77]:
# Run this block to set up matrix for gauss seidel and jacobi iteration


n = 50
m = n
# Should be diagonally dominant
# Matrix is randomized with zeroes and ones
# The maximum non diagonal sum in each row is n * 1 - 1
max_non_diag = (n * 1) - 1
# This means we must add an identity matrix with at least n * 1 - 1
# on the diagonal according to example 7.6
A = np.random.randint(2, size=(m, n)) + max_non_diag * np.eye(n)
y = np.random.randint(3, size=n)
b = A.dot(y)

print("Input matrix = ", A)
print("Manufactured solution y = ", y)
print("b = ", b)

Input matrix =  [[50.  0.  0. ...  1.  1.  1.]
 [ 0. 49.  1. ...  1.  0.  1.]
 [ 0.  1. 49. ...  0.  1.  1.]
 ...
 [ 0.  0.  0. ... 49.  1.  1.]
 [ 1.  1.  0. ...  0. 49.  0.]
 [ 1.  1.  1. ...  1.  1. 49.]]
Manufactured solution y =  [2 1 1 0 0 1 0 0 1 0 0 1 0 2 0 0 2 1 2 1 0 1 0 2 2 0 2 0 0 0 0 2 1 1 1 1 2
 2 1 1 0 0 1 1 0 0 1 0 1 1]
b =  [122.  66.  66.  24.  20.  72.  15.  17.  66.  25.  20.  76.  25. 115.
  23.  20. 116.  69. 116.  72.  20.  67.  19. 120. 120.  22. 110.  14.
  16.  21.  24. 121.  75.  68.  69.  67. 117. 118.  70.  69.  23.  20.
  76.  66.  17.  21.  72.  19.  63.  67.]


In [78]:
# Assignment 1
# Jacobi iteration for Ax = b
# Jacobi iteration is based on splitting the A = A_1 + A_2 where
# A_1 = D = diag(A) and A_2 = A - D
# Example 7.8 used from the book

def jacobi_iteration(A, b):
  m, n = np.shape(A)
  x = np.zeros(n)
  k = 20
  x_matrix = np.zeros((k, len(x)))
  r_vec = np.zeros(k)
  for iter in range(k):
    x_k = x
    for i in range(0, n):
      sum = 0
      for j in range(0, n):
        if j == i:
          sum = sum
        else:
          sum += A[i][j]*x_k[j]
      x[i] = 1/A[i][i] * (b[i] - sum)
    x_matrix[iter,:] = x
    r = np.linalg.norm(A.dot(x)-b)
    r_vec[iter] = r
  return(x, x_matrix, r_vec)

D = np.diag(np.diag(A))
D_inv = np.linalg.inv(D)
Mj = np.eye(n) - np.matmul(D_inv, A)
conv_criterion = np.linalg.norm(Mj)

print("M = ", Mj)
print("Convergence criterion = ", conv_criterion)

x, x_matrix, r = jacobi_iteration(A, b)

error_matrix = x_matrix - y
error_vec = np.zeros_like(r)
i = 0
for error in error_matrix:
  error_vec[i] = np.linalg.norm(error)
  i += 1

print("Error from manufactured solution y = ",error_vec)
print("Result x = ", x)
print("Result residuals r = ", r)

error_vec_jacobi = error_vec
residuals_jacobi = r

M =  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -2.00000000e-02
  -2.00000000e-02 -2.00000000e-02]
 [ 0.00000000e+00  1.11022302e-16 -2.04081633e-02 ... -2.04081633e-02
   0.00000000e+00 -2.04081633e-02]
 [ 0.00000000e+00 -2.04081633e-02  1.11022302e-16 ...  0.00000000e+00
  -2.04081633e-02 -2.04081633e-02]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.11022302e-16
  -2.04081633e-02 -2.04081633e-02]
 [-2.04081633e-02 -2.04081633e-02  0.00000000e+00 ...  0.00000000e+00
   1.11022302e-16  0.00000000e+00]
 [-2.04081633e-02 -2.04081633e-02 -2.04081633e-02 ... -2.04081633e-02
  -2.04081633e-02  1.11022302e-16]]
Convergence criterion =  0.7185658352870782
Error from manufactured solution y =  [1.50960609e+00 1.88841250e-01 1.27223745e-02 2.09842799e-03
 2.13895285e-04 1.90283007e-05 3.01995731e-06 2.25309408e-07
 3.22487800e-08 3.66089765e-09 2.91301890e-10 4.85736885e-11
 3.87073018e-12 4.90766278e-13 6.17235242e-14 4.69652179e-15
 9.74719069e-16 6.32208300e-16 6.

In [79]:
# Assignment 2
# Based on splitting into A_1 = L and A_2 = A - L
# Example 7.9 used from the book

def gauss_seidel_iteration(A, b):
  m, n = np.shape(A)
  x = np.zeros(n)
  k = 20
  x_matrix = np.zeros((k, len(x)))
  r_vec = np.zeros(k)
  for iter in range(k):
    for i in range(0, n):
      sum_1 = 0
      sum_2 = 0
      for j in range(0, i):
        sum_1 += A[i][j]*x[j]
      for j in range(i+1, n):
        sum_2 += A[i][j]*x[j]
      x[i] = 1/A[i][i] * (b[i] - sum_1 - sum_2)
    x_matrix[iter,:] = x
    r = np.linalg.norm(A.dot(x)-b)
    r_vec[iter] = r
  return(x, x_matrix, r_vec)

L = np.tril(A)
L_inv = np.linalg.inv(L)
Mj = np.eye(n) - np.matmul(L_inv, A)
conv_criterion = np.linalg.norm(Mj)

print("M = ", Mj)
print("Convergence criterion = ", conv_criterion)

x, x_matrix, r_vec = gauss_seidel_iteration(A, b)

error_matrix = x_matrix - y
error_vec = np.zeros_like(r)
i = 0
for error in error_matrix:
  error_vec[i] = np.linalg.norm(error)
  i += 1

print("Error from manufactured solution y = ",error_vec)
print("Result x = ", x)
print("Result residuals r = ", r)

error_vec_gauss_seidel = error_vec
residuals_gauss_seidel = r


M =  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -2.00000000e-02
  -2.00000000e-02 -2.00000000e-02]
 [ 0.00000000e+00  1.11022302e-16 -2.04081633e-02 ... -2.04081633e-02
   0.00000000e+00 -2.04081633e-02]
 [ 0.00000000e+00 -3.46944695e-18  4.16493128e-04 ...  4.16493128e-04
  -2.04081633e-02 -1.99916701e-02]
 ...
 [-1.19262239e-18 -5.42101086e-19 -1.12403648e-04 ...  2.91489820e-03
  -1.37787764e-02 -1.44343733e-02]
 [-3.46944695e-18  0.00000000e+00  3.61379298e-04 ...  3.00194883e-03
   4.83207255e-03  4.13692570e-03]
 [ 0.00000000e+00  0.00000000e+00  3.26167400e-04 ...  4.76487649e-03
   4.45151601e-03  4.18973446e-03]]
Convergence criterion =  0.48009886504467425
Error from manufactured solution y =  [1.50960609e+00 1.88841250e-01 1.27223745e-02 2.09842799e-03
 2.13895285e-04 1.90283007e-05 3.01995731e-06 2.25309408e-07
 3.22487800e-08 3.66089754e-09 2.91301809e-10 4.85736659e-11
 3.87054729e-12 4.90523836e-13 6.16782515e-14 4.75159414e-15
 8.45956043e-16 6.28201942e-16 6

# **Results**

In [ ]:
# Plot results



Results here

# **Discussion**

Discussion here